# Exportar Dados para Web

Exporta os dados processados do ETL para formato JSON compatível com o site de visualização.

**⚠️ IMPORTANTE: Este notebook exporta TODOS os crimes individuais, não agregados!**

Objetivos:
- Carregar dados consolidados com coordenadas geográficas válidas
- Manter cada crime como registro individual (não agregar)
- Exportar TODAS as informações relevantes de cada crime
- Gerar arquivos JSON otimizados para visualização web
- Criar estatísticas gerais
- Gerar amostra estratificada para testes rápidos

Dados exportados por crime:
- Localização: município, bairro, endereço, coordenadas, tipo de local
- Crime: tipo, categoria, rubrica, natureza apurada
- Temporal: data, hora, dia da semana, período do dia
- Registro: delegacia, número do BO, ano
- Site exibe popup detalhado ao clicar em cada marcador

In [10]:
import pandas as pd
import json
from pathlib import Path

print("Bibliotecas importadas")

Bibliotecas importadas


## 1. Importar Bibliotecas

Importação das bibliotecas necessárias para exportação de dados para formato web. As bibliotecas utilizadas incluem:

- **pandas**: manipulação e filtragem dos dados processados
- **json**: serialização dos dados para formato JSON compatível com JavaScript
- **pathlib**: gerenciamento de caminhos de arquivos multiplataforma

Este notebook converte dados processados em formato Parquet para JSON otimizado, pronto para consumo pela interface web de visualização de criminalidade.

In [11]:
PROJECT_ROOT = Path().resolve().parent
DATA_PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
SITE_DIR = PROJECT_ROOT / 'site'

print(f"Diretório processados: {DATA_PROCESSED_DIR}")
print(f"Diretório site: {SITE_DIR}")

Diretório processados: C:\Users\jodes\OneDrive\Documentos\Projetos-GitHub\public-security-data\data\processed
Diretório site: C:\Users\jodes\OneDrive\Documentos\Projetos-GitHub\public-security-data\site


## 2. Configurar Diretórios

Definição dos caminhos para os diretórios do projeto:

- **data/processed/**: contém os dados transformados em formato Parquet
- **site/**: destino dos arquivos JSON para a aplicação web

Esta organização mantém separação clara entre dados processados (backend) e recursos web (frontend), facilitando deploy e manutenção da aplicação de visualização.

In [12]:
# Carregar dados transformados (após filtragem no notebook 2)
arquivo_principal = DATA_PROCESSED_DIR / 'ocorrencias_criminais_2025_transformado.parquet'

if arquivo_principal.exists():
    df = pd.read_parquet(arquivo_principal)
    print(f"Dados carregados de: {arquivo_principal.name}")
    print(f"Registros totais: {len(df):,}")
    
    # Filtrar apenas registros válidos
    # 1. Coordenadas válidas e não-zeradas
    df = df[
        (df['latitude'].notna()) & 
        (df['longitude'].notna()) & 
        (df['latitude'] != 0.0) & 
        (df['longitude'] != 0.0)
    ]
    print(f"Registros com coordenadas válidas: {len(df):,}")
    
    # 2. Apenas dados de 2025 (redundante se notebook 2 foi executado, mas garante)
    if 'ano_bo' in df.columns:
        df = df[df['ano_bo'] >= 2025]
        print(f"Registros de 2025: {len(df):,}")
    
    print(f"\nColunas disponíveis ({len(df.columns)}):")
    print(df.columns.tolist())
    print(f"\nAmostra dos dados:")
    display(df.head(3))
else:
    print("Arquivo não encontrado. Execute os notebooks anteriores:")
    print("  1. 1_extracao.ipynb")
    print("  2. 2_transformacao.ipynb (com filtros de qualidade)")
    df = None

Dados carregados de: ocorrencias_criminais_2025_transformado.parquet
Registros totais: 878,585
Registros com coordenadas válidas: 637,016
Registros de 2025: 637,016

Colunas disponíveis (36):
['nome_departamento', 'nome_seccional', 'nome_delegacia', 'nome_municipio', 'num_bo', 'ano_bo', 'data_registro', 'data_ocorrencia_bo', 'hora_ocorrencia_bo', 'desc_periodo', 'descr_subtipolocal', 'bairro', 'logradouro', 'numero_logradouro', 'latitude', 'longitude', 'nome_delegacia_circunscricao', 'nome_departamento_circunscricao', 'nome_seccional_circunscricao', 'nome_municipio_circunscricao', 'rubrica', 'descr_conduta', 'natureza_apurada', 'mes_estatistica', 'ano_estatistica', 'cmd', 'btl', 'cia', 'aba_origem', 'ano_mes', 'ano_ocorrencia', 'mes_ocorrencia', 'dia_semana', 'dia_semana_nome', 'tipo_crime', 'categoria_crime']

Amostra dos dados:
Registros de 2025: 637,016

Colunas disponíveis (36):
['nome_departamento', 'nome_seccional', 'nome_delegacia', 'nome_municipio', 'num_bo', 'ano_bo', 'data_re

,nome_departamento,nome_seccional,nome_delegacia,nome_municipio,num_bo,ano_bo,data_registro,data_ocorrencia_bo,hora_ocorrencia_bo,desc_periodo,...,btl,cia,aba_origem,ano_mes,ano_ocorrencia,mes_ocorrencia,dia_semana,dia_semana_nome,tipo_crime,categoria_crime
4,DIPOL - DEPTO DE INTELIGENCIA,DELEGACIA ELETRONICA,DELEGACIA ELETRONICA 1,S.PAULO,AA1448,2025,2025-01-01,2024-12-31,10:30:00,nan,...,7ºBPM/M,3ªCIA 4ªCIA,JAN_A_JUN_2025,NaT,2024.0,12.0,1.0,Tuesday,FURTO - OUTROS,Crimes Patrimoniais
5,DIPOL - DEPTO DE INTELIGENCIA,DELEGACIA ELETRONICA,DELEGACIA ELETRONICA 1,S.PAULO,AA1473,2025,2025-01-01,2024-12-31,23:30:00,nan,...,7ºBPM/M,3ªCIA 4ªCIA,JAN_A_JUN_2025,NaT,2024.0,12.0,1.0,Tuesday,FURTO - OUTROS,Crimes Patrimoniais
6,DIPOL - DEPTO DE INTELIGENCIA,DELEGACIA ELETRONICA,DELEGACIA ELETRONICA 1,S.PAULO,AA2036,2025,2025-01-01,2024-12-31,21:30:00,nan,...,7ºBPM/M,3ªCIA 4ªCIA,JAN_A_JUN_2025,NaT,2024.0,12.0,1.0,Tuesday,FURTO - OUTROS,Crimes Patrimoniais


## 3. Carregar Dados com Coordenadas

Carregamento dos dados transformados com filtros adicionais para garantir qualidade na visualização web:

1. **Filtro de coordenadas**: remove registros sem geolocalização ou com coordenadas (0.0, 0.0) inválidas
2. **Filtro temporal**: mantém apenas dados de 2025, conforme escopo do projeto
3. **Validação de completude**: verifica disponibilidade das colunas necessárias

Estes filtros garantem que apenas crimes geolocalizados e relevantes sejam exibidos no mapa interativo, melhorando a experiência do usuário e a utilidade das visualizações.

In [13]:
if df is not None:
    # Selecionar colunas relevantes para o site
    colunas_export = [
        'latitude', 'longitude',
        'rubrica', 'tipo_crime', 'categoria_crime',
        'nome_municipio_circunscricao', 'bairro',
        'logradouro', 'numero_logradouro',
        'data_ocorrencia_bo', 'hora_ocorrencia_bo',
        'desc_periodo', 'descr_subtipolocal',
        'nome_delegacia_circunscricao',
        'ano_bo', 'mes_estatistica',
        'dia_semana_nome', 'num_bo',
        'natureza_apurada'
    ]
    
    # Verificar quais colunas existem
    colunas_presentes = [col for col in colunas_export if col in df.columns]
    print(f"Colunas selecionadas para exportação: {len(colunas_presentes)}")
    print(colunas_presentes)
    
    # Criar DataFrame de exportação
    df_export = df[colunas_presentes].copy()
    
    # Renomear colunas para formato web-friendly
    rename_map = {
        'latitude': 'lat',
        'longitude': 'lng',
        'nome_municipio_circunscricao': 'municipio',
        'mes_estatistica': 'mes',
        'ano_bo': 'ano',
        'data_ocorrencia_bo': 'data_ocorrencia',
        'hora_ocorrencia_bo': 'hora_ocorrencia',
        'nome_delegacia_circunscricao': 'delegacia',
        'dia_semana_nome': 'dia_semana'
    }
    
    df_export = df_export.rename(columns={k: v for k, v in rename_map.items() if k in df_export.columns})
    
    # CORREÇÃO: Substituir NaN e valores infinitos por None/null
    import numpy as np
    df_export = df_export.replace([np.inf, -np.inf], None)
    df_export = df_export.where(pd.notna(df_export), None)
    
    # Limpar valores nulos em campos de texto (converter None para string vazia)
    for col in df_export.select_dtypes(include=['object']).columns:
        df_export[col] = df_export[col].fillna('').astype(str)
        df_export[col] = df_export[col].replace('None', '')
        df_export[col] = df_export[col].replace('nan', '')
    
    # Converter datas para string
    if 'data_ocorrencia' in df_export.columns:
        df_export['data_ocorrencia'] = pd.to_datetime(df_export['data_ocorrencia']).dt.strftime('%Y-%m-%d')
    
    print(f"\nDados preparados para exportação")
    print(f"Total de crimes individuais: {len(df_export):,}")
    print(f"\nColunas finais: {list(df_export.columns)}")
    print(f"\nExemplo dos dados:")
    display(df_export.head(3))
else:
    print("Nenhum dado para processar")
    df_export = None

Colunas selecionadas para exportação: 19
['latitude', 'longitude', 'rubrica', 'tipo_crime', 'categoria_crime', 'nome_municipio_circunscricao', 'bairro', 'logradouro', 'numero_logradouro', 'data_ocorrencia_bo', 'hora_ocorrencia_bo', 'desc_periodo', 'descr_subtipolocal', 'nome_delegacia_circunscricao', 'ano_bo', 'mes_estatistica', 'dia_semana_nome', 'num_bo', 'natureza_apurada']

Dados preparados para exportação
Total de crimes individuais: 637,016

Colunas finais: ['lat', 'lng', 'rubrica', 'tipo_crime', 'categoria_crime', 'municipio', 'bairro', 'logradouro', 'numero_logradouro', 'data_ocorrencia', 'hora_ocorrencia', 'desc_periodo', 'descr_subtipolocal', 'delegacia', 'ano', 'mes', 'dia_semana', 'num_bo', 'natureza_apurada']

Exemplo dos dados:

Dados preparados para exportação
Total de crimes individuais: 637,016

Colunas finais: ['lat', 'lng', 'rubrica', 'tipo_crime', 'categoria_crime', 'municipio', 'bairro', 'logradouro', 'numero_logradouro', 'data_ocorrencia', 'hora_ocorrencia', 'desc

,lat,lng,rubrica,tipo_crime,categoria_crime,municipio,bairro,logradouro,numero_logradouro,data_ocorrencia,hora_ocorrencia,desc_periodo,descr_subtipolocal,delegacia,ano,mes,dia_semana,num_bo,natureza_apurada
4,-23.552842,-46.627423,Furto (art. 155),FURTO - OUTROS,Crimes Patrimoniais,S.PAULO,SE,AVENIDA PREFEITO PASSOS,57.0,2024-12-31,10:30:00,,Via Pública,01º D.P. SE,2025,1,Tuesday,AA1448,FURTO - OUTROS
5,-23.556547,-46.629346,Furto (art. 155),FURTO - OUTROS,Crimes Patrimoniais,S.PAULO,LIBERDADE,RUA GLICÉRIO,437.0,2024-12-31,23:30:00,,Via Pública,01º D.P. SE,2025,1,Tuesday,AA1473,FURTO - OUTROS
6,-23.555854,-46.629199,Furto (art. 155),FURTO - OUTROS,Crimes Patrimoniais,S.PAULO,LIBERDADE,RUA GLICÉRIO,368.0,2024-12-31,21:30:00,,Via Pública,01º D.P. SE,2025,1,Tuesday,AA2036,FURTO - OUTROS


## 4. Preparar Dados para Exportação

Preparação dos dados para formato JSON compatível com a aplicação web. Este processo inclui:

1. **Seleção de colunas**: apenas campos relevantes para visualização são mantidos
2. **Renomeação**: colunas recebem nomes simplificados e web-friendly (ex: `nome_municipio_circunscricao` vira `municipio`)
3. **Limpeza de valores nulos**: tratamento de NaN, infinitos e valores inválidos que quebrariam o JSON
4. **Conversão de tipos**: datas convertidas para strings formatadas, coordenadas arredondadas
5. **Validação**: verificação de que todos os valores são serializáveis em JSON

Esta preparação garante que o arquivo JSON gerado seja válido, compacto e otimizado para carregamento rápido no navegador.

In [14]:
if df_export is not None and len(df_export) > 0:
    df_export_clean = df_export.copy()
    
    # Converter tipos de dados para JSON-friendly
    for col in ['lat', 'lng']:
        if col in df_export_clean.columns:
            df_export_clean[col] = df_export_clean[col].astype(float).round(6)
    
    for col in ['mes', 'ano']:
        if col in df_export_clean.columns:
            # Converter para int, mas manter None se for nulo
            df_export_clean[col] = df_export_clean[col].apply(lambda x: int(x) if pd.notna(x) else None)
    
    # Converter para lista de dicionários (orient='records' já lida bem com None)
    dados_json = df_export_clean.to_dict('records')
    
    print(f"Dados convertidos para JSON")
    print(f"Total de crimes: {len(dados_json):,}")
    print(f"\nExemplo dos primeiros 2 registros:")
    for i, registro in enumerate(dados_json[:2], 1):
        print(f"\nCrime {i}:")
        print(json.dumps(registro, indent=2, ensure_ascii=False))
else:
    print("Nenhum dado para converter")
    dados_json = []

Dados convertidos para JSON
Total de crimes: 637,016

Exemplo dos primeiros 2 registros:

Crime 1:
{
  "lat": -23.552842,
  "lng": -46.627423,
  "rubrica": "Furto (art. 155)",
  "tipo_crime": "FURTO - OUTROS",
  "categoria_crime": "Crimes Patrimoniais",
  "municipio": "S.PAULO",
  "bairro": "SE",
  "logradouro": "AVENIDA PREFEITO PASSOS",
  "numero_logradouro": 57.0,
  "data_ocorrencia": "2024-12-31",
  "hora_ocorrencia": "10:30:00",
  "desc_periodo": "",
  "descr_subtipolocal": "Via Pública",
  "delegacia": "01º D.P. SE",
  "ano": 2025,
  "mes": 1,
  "dia_semana": "Tuesday",
  "num_bo": "AA1448",
  "natureza_apurada": "FURTO - OUTROS"
}

Crime 2:
{
  "lat": -23.556547,
  "lng": -46.629346,
  "rubrica": "Furto (art. 155)",
  "tipo_crime": "FURTO - OUTROS",
  "categoria_crime": "Crimes Patrimoniais",
  "municipio": "S.PAULO",
  "bairro": "LIBERDADE",
  "logradouro": "RUA GLICÉRIO",
  "numero_logradouro": 437.0,
  "data_ocorrencia": "2024-12-31",
  "hora_ocorrencia": "23:30:00",
  "desc_

## 5. Converter para JSON

Conversão do DataFrame preparado para estrutura de lista de dicionários, formato ideal para serialização JSON. Cada registro representa um crime individual com todas suas propriedades. A conversão utiliza `to_dict('records')` que gera estrutura otimizada para JavaScript, onde cada objeto pode ser facilmente manipulado no frontend. Exemplos dos primeiros registros são exibidos para validação visual da estrutura antes do salvamento.

In [15]:
if 'dados_json' in locals() and len(dados_json) > 0:
    json_path = SITE_DIR / 'dados_criminais.json'
    
    # Salvar com allow_nan=False para garantir que não há NaN no JSON
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(dados_json, f, ensure_ascii=False, indent=2, allow_nan=False)
    
    size_mb = json_path.stat().st_size / (1024 * 1024)
    
    print(f"Arquivo JSON salvo com sucesso")
    print(f"Caminho: {json_path}")
    print(f"Tamanho: {size_mb:.2f} MB")
    print(f"Registros: {len(dados_json):,}")
    
    if size_mb > 10:
        print(f"\nAviso: Arquivo grande ({size_mb:.2f} MB). Considere usar amostragem para o site.")
else:
    print("Nenhum dado para salvar")

ValueError: Out of range float values are not JSON compliant: nan

## 6. Salvar Arquivo JSON Completo

Persistência do arquivo JSON contendo todos os crimes geolocalizados. O arquivo é salvo com:

- **Encoding UTF-8**: suporte completo a caracteres especiais e acentuação
- **Indentação**: formato legível para debugging (indent=2)
- **allow_nan=False**: validação rigorosa que impede valores inválidos NaN
- **ensure_ascii=False**: preserva caracteres Unicode originais

O tamanho do arquivo é reportado para monitoramento. Arquivos muito grandes (> 10 MB) podem impactar performance no navegador, justificando a criação de arquivos de amostra menores para testes.

In [ ]:
if df_export is not None and len(df_export) > 0:
    # Gerar estatísticas
    stats = {
        'total_crimes': int(len(df_export)),
        'data_geracao': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    # Estatísticas por categoria
    if 'categoria_crime' in df_export.columns:
        stats['categorias'] = df_export['categoria_crime'].value_counts().to_dict()
    
    if 'tipo_crime' in df_export.columns:
        stats['tipos_crime_unicos'] = int(df_export['tipo_crime'].nunique())
        stats['top_crimes'] = df_export['tipo_crime'].value_counts().head(15).to_dict()
    
    if 'municipio' in df_export.columns:
        stats['municipios_unicos'] = int(df_export['municipio'].nunique())
        stats['top_municipios'] = df_export['municipio'].value_counts().head(15).to_dict()
    
    if 'bairro' in df_export.columns:
        bairros = df_export[df_export['bairro'] != '']['bairro']
        if len(bairros) > 0:
            stats['top_bairros'] = bairros.value_counts().head(15).to_dict()
    
    if 'ano' in df_export.columns and 'mes' in df_export.columns:
        stats['periodo'] = {
            'ano_inicio': int(df_export['ano'].min()),
            'ano_fim': int(df_export['ano'].max()),
            'mes_inicio': int(df_export['mes'].min()),
            'mes_fim': int(df_export['mes'].max())
        }
    
    if 'dia_semana' in df_export.columns:
        stats['crimes_por_dia_semana'] = df_export['dia_semana'].value_counts().to_dict()
    
    # Salvar estatísticas
    stats_path = SITE_DIR / 'estatisticas.json'
    with open(stats_path, 'w', encoding='utf-8') as f:
        json.dump(stats, f, indent=2, ensure_ascii=False)
    
    print(f"Estatísticas salvas: {stats_path}")
    print(f"Tamanho: {stats_path.stat().st_size / 1024:.1f} KB")
    print(f"\nResumo:")
    print(f"  Total de crimes: {stats['total_crimes']:,}")
    if 'tipos_crime_unicos' in stats:
        print(f"  Tipos de crime: {stats['tipos_crime_unicos']}")
    if 'municipios_unicos' in stats:
        print(f"  Municípios: {stats['municipios_unicos']}")
else:
    print("Nenhum dado para gerar estatísticas")

Estatísticas salvas: C:\Users\jodes\OneDrive\Documentos\Projetos-GitHub\public-security-data\site\estatisticas.json
Tamanho: 1.9 KB

Resumo:
  Total de crimes: 637,016
  Tipos de crime: 22
  Municípios: 643


## 7. Criar Estatísticas para o Site

Geração de arquivo separado contendo estatísticas agregadas para exibição na interface web. Este arquivo `estatisticas.json` inclui:

- Total de crimes e timestamp de geração
- Distribuição por categorias de crime
- Top 15 tipos de crime mais frequentes
- Top 15 municípios com mais ocorrências
- Top 15 bairros (quando disponível)
- Período temporal coberto pelos dados
- Crimes por dia da semana

Estas estatísticas pré-computadas melhoram drasticamente a performance do site, evitando cálculos pesados no navegador do usuário.

In [ ]:
if df_export is not None and len(df_export) > 0:
    # Criar amostra estratificada para testes rápidos
    sample_size = min(5000, len(df_export))
    
    # Se houver tipo_crime, fazer amostragem estratificada
    if 'tipo_crime' in df_export.columns:
        # Amostra proporcional por tipo de crime
        df_sample = df_export.groupby('tipo_crime', group_keys=False).apply(
            lambda x: x.sample(min(len(x), max(1, int(sample_size * len(x) / len(df_export)))))
        ).reset_index(drop=True)
    else:
        # Amostra aleatória simples
        df_sample = df_export.sample(n=sample_size, random_state=42)
    
    dados_sample_json = df_sample.to_dict('records')
    
    sample_path = SITE_DIR / 'dados_criminais_sample.json'
    # Salvar com allow_nan=False para garantir que não há NaN no JSON
    with open(sample_path, 'w', encoding='utf-8') as f:
        json.dump(dados_sample_json, f, ensure_ascii=False, indent=2, allow_nan=False)
    
    print(f"Arquivo de amostra criado")
    print(f"Caminho: {sample_path}")
    print(f"Crimes na amostra: {len(dados_sample_json):,}")
    print(f"Tamanho: {sample_path.stat().st_size / (1024*1024):.2f} MB")
    print(f"\nCritério: Amostra estratificada por tipo de crime")
    
    if 'tipo_crime' in df_sample.columns:
        print(f"\nDistribuição na amostra:")
        print(df_sample['tipo_crime'].value_counts().head(10))
else:
    print("Nenhum dado para criar amostra")

Arquivo de amostra criado
Caminho: C:\Users\jodes\OneDrive\Documentos\Projetos-GitHub\public-security-data\site\dados_criminais_sample.json
Crimes na amostra: 4,996
Tamanho: 3.12 MB

Critério: Amostra estratificada por tipo de crime

Distribuição na amostra:
tipo_crime
FURTO - OUTROS                                     2389
ROUBO - OUTROS                                      820
FURTO DE VEÍCULO                                    424
LESÃO CORPORAL DOLOSA                               418
LESÃO CORPORAL CULPOSA POR ACIDENTE DE TRÂNSITO     398
TRÁFICO DE ENTORPECENTES                            209
ROUBO DE VEÍCULO                                    132
PORTE DE ENTORPECENTES                               67
APREENSÃO DE ENTORPECENTES                           31
HOMICÍDIO CULPOSO POR ACIDENTE DE TRÂNSITO           22
Name: count, dtype: int64


C:\Users\jodes\AppData\Local\Temp\ipykernel_24920\3095378290.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sample = df_export.groupby('tipo_crime', group_keys=False).apply(


## 8. Criar Arquivo de Amostra

Geração de arquivo JSON reduzido contendo amostra estratificada de 5.000 crimes para testes e desenvolvimento. A amostragem estratificada garante que:

- A proporção de cada tipo de crime é mantida
- Todos os tipos principais estão representados
- O arquivo é suficientemente pequeno para carregamento rápido
- A amostra é reprodutível (random_state fixo)

Este arquivo `dados_criminais_sample.json` é carregado por padrão na aplicação web, permitindo testes rápidos sem impactar performance. Para visualizar todos os crimes, o usuário pode alternar para o arquivo completo através da interface.

In [ ]:
print("="*70)
print("RESUMO DA EXPORTAÇÃO")
print("="*70)

print("\nArquivos gerados no diretório site/:")

if SITE_DIR.exists():
    for file in sorted(SITE_DIR.glob('*.json')):
        size = file.stat().st_size
        if size > 1024*1024:
            size_str = f"{size/(1024*1024):.2f} MB"
        else:
            size_str = f"{size/1024:.2f} KB"
        print(f"  {file.name:<40} ({size_str})")

if df_export is not None:
    print(f"\n{'='*70}")
    print("INFORMAÇÕES DOS DADOS EXPORTADOS")
    print(f"{'='*70}")
    print(f"\nTotal de crimes individuais: {len(df_export):,}")
    print(f"Campos exportados por crime: {len(df_export.columns)}")
    print(f"\nCampos disponíveis em cada crime:")
    for col in df_export.columns:
        print(f"  • {col}")

print("\n" + "="*70)
print("EXPORTAÇÃO CONCLUÍDA COM SUCESSO")
print("="*70)

print("\n🗺️ VISUALIZAR O MAPA:")
print(f"  1. Abra o arquivo: {SITE_DIR / 'main.html'}")
print("  2. Use um navegador web moderno (Chrome, Firefox, Edge)")
print("  3. Clique nos marcadores para ver TODOS os detalhes de cada crime")

print("\n📊 RECURSOS DO SITE:")
print("  • Cada marcador = 1 crime individual")
print("  • Popups com informações completas:")
print("    - Tipo e categoria do crime")
print("    - Localização detalhada (município, bairro, endereço)")
print("    - Data, hora e dia da semana")
print("    - Delegacia e número do boletim")
print("    - Coordenadas geográficas")
print("  • Filtros por tipo de crime e município")
print("  • Clustering automático para melhor visualização")

print("\n💡 DICA:")
print("  O site carrega 'dados_criminais_sample.json' por padrão (5.000 crimes)")
print("  Para carregar todos os crimes, altere no leaflet.js:")
print("    toggleDataSource(false) // carrega dados_criminais.json")

print("\n" + "="*70)

RESUMO DA EXPORTAÇÃO

Arquivos gerados no diretório site/:
  dados_criminais.json                     (393.13 MB)
  dados_criminais_sample.json              (3.12 MB)
  estatisticas.json                        (1.93 KB)

INFORMAÇÕES DOS DADOS EXPORTADOS

Total de crimes individuais: 637,016
Campos exportados por crime: 19

Campos disponíveis em cada crime:
  • lat
  • lng
  • rubrica
  • tipo_crime
  • categoria_crime
  • municipio
  • bairro
  • logradouro
  • numero_logradouro
  • data_ocorrencia
  • hora_ocorrencia
  • desc_periodo
  • descr_subtipolocal
  • delegacia
  • ano
  • mes
  • dia_semana
  • num_bo
  • natureza_apurada

EXPORTAÇÃO CONCLUÍDA COM SUCESSO

🗺️ VISUALIZAR O MAPA:
  1. Abra o arquivo: C:\Users\jodes\OneDrive\Documentos\Projetos-GitHub\public-security-data\site\main.html
  2. Use um navegador web moderno (Chrome, Firefox, Edge)
  3. Clique nos marcadores para ver TODOS os detalhes de cada crime

📊 RECURSOS DO SITE:
  • Cada marcador = 1 crime individual
  • Popu

## 9. Resumo da Exportação

Sumário executivo final do processo de exportação para web, consolidando todas as informações relevantes:

### Arquivos Gerados

Lista completa dos arquivos JSON criados no diretório `site/`, incluindo tamanhos

### Estrutura dos Dados

Documentação dos campos disponíveis em cada registro de crime, facilitando integração frontend

### Instruções de Uso

Orientações claras para visualizar a aplicação web:
- Como abrir o mapa interativo
- Recursos disponíveis (filtros, clustering, popups detalhados)
- Alternância entre arquivo completo e amostra

Este resumo serve como documentação final do pipeline completo, desde extração até visualização web, encerrando o ciclo ETL do projeto de segurança pública de São Paulo.